In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# training_data = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-training-data.csv', delimiter=';')
# training_data
test_data = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-test-data.csv', delimiter=';')
test_data

,obs_id,SiteId,Timestamp,ForecastId,Value
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409
4,6793052,235,2014-01-13T14:15:00+00:00,5005,91885.429363
...,...,...,...,...,...
1309171,268524,261,2014-03-24T03:00:00+00:00,5487,38545.410018
1309172,3203969,261,2014-03-24T12:30:00+00:00,5487,48937.146780
1309173,7721495,261,2014-03-24T22:00:00+00:00,5487,NaN
1309174,5696668,261,2014-03-24T23:30:00+00:00,5487,NaN


In [3]:
weather_data = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-weather.csv', delimiter=';')
weather_data

,Timestamp,Temperature,Distance,SiteId
0,2017-03-03T19:00:00+00:00,10.6,27.489346,51
1,2017-03-03T19:20:00+00:00,11.0,28.663082,51
2,2017-03-03T20:00:00+00:00,6.3,28.307039,51
3,2017-03-03T21:55:00+00:00,10.0,29.797449,51
4,2017-03-03T23:00:00+00:00,5.4,28.307039,51
...,...,...,...,...
3957030,2016-09-11T11:00:00+00:00,25.9,28.307039,51
3957031,2016-09-11T11:20:00+00:00,27.0,27.489346,51
3957032,2016-09-11T12:00:00+00:00,27.1,28.307039,51
3957033,2016-09-11T15:50:00+00:00,28.0,27.489346,51


In [4]:
forecast_data = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-submission-forecast-period.csv', delimiter=';')
forecast_data

,ForecastId,ForecastPeriodNS
0,123,900000000000
1,264,900000000000
2,596,900000000000
3,914,900000000000
4,1053,900000000000
...,...,...
6969,6387,900000000000
6970,6487,3600000000000
6971,6569,900000000000
6972,6777,900000000000


In [5]:
metadata = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-metadata.csv', delimiter=';')
metadata

,SiteId,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff
0,207,7964.873347,30.0,18.0,False,False,False,False,False,True,True
1,7,15168.125971,30.0,18.0,False,False,False,False,False,True,True
2,74,424.340663,15.0,18.0,False,False,False,False,False,True,True
3,239,1164.822636,15.0,18.0,False,False,False,False,False,True,True
4,274,1468.246690,5.0,18.0,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...
262,192,11188.881545,15.0,18.0,False,False,False,False,False,True,True
263,58,1149.050606,15.0,18.0,False,False,False,False,False,True,True
264,123,5470.205018,15.0,18.0,False,False,False,False,False,True,True
265,122,6843.612340,15.0,18.0,False,False,False,False,False,True,True


In [6]:
holiday_data = pd.read_csv(f'../dataset/power-laws-forecasting-energy-consumption-holidays.csv', delimiter=';')
holiday_data

,Date,Holiday,SiteId
0,2016-02-15,Washington's Birthday,1
1,2017-05-29,Memorial Day,1
2,2017-11-23,Thanksgiving Day,1
3,2017-12-29,New Years Eve Shift,1
4,2017-12-31,New Years Eve,1
...,...,...,...
8382,2015-12-26,Boxing Day,303
8383,2016-05-01,International Workers' Day,304
8384,2015-04-25,Liberation Day,304
8385,2016-03-28,Easter Monday,305


In [7]:
# data = pd.merge(training_data, metadata, on='SiteId')
data = pd.merge(test_data, metadata, on='SiteId')
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True
4,6793052,235,2014-01-13T14:15:00+00:00,5005,91885.429363,10906.977168,15.0,18.0,False,False,False,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7707783,174,2017-11-30T19:45:00+00:00,3510,29927.876062,587.763238,15.0,18.0,False,False,False,False,False,True,True
1309172,1873823,174,2017-11-30T21:45:00+00:00,3510,15303.633895,587.763238,15.0,18.0,False,False,False,False,False,True,True
1309173,7376879,174,2017-12-03T06:45:00+00:00,3510,12752.003943,587.763238,15.0,18.0,False,False,False,False,False,True,True
1309174,1259696,174,2017-12-03T15:45:00+00:00,3510,13869.983259,587.763238,15.0,18.0,False,False,False,False,False,True,True


In [8]:
data = pd.merge(data, forecast_data, on='ForecastId')
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,ForecastPeriodNS
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000
4,6124150,235,2014-01-01T15:45:00+00:00,5004,153731.391434,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7707783,174,2017-11-30T19:45:00+00:00,3510,29927.876062,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000
1309172,1873823,174,2017-11-30T21:45:00+00:00,3510,15303.633895,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000
1309173,7376879,174,2017-12-03T06:45:00+00:00,3510,12752.003943,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000
1309174,1259696,174,2017-12-03T15:45:00+00:00,3510,13869.983259,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000


In [9]:
data['Date'] = pd.to_datetime(data['Timestamp']).dt.date
data


,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,ForecastPeriodNS,Date
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-03
4,6124150,235,2014-01-01T15:45:00+00:00,5004,153731.391434,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7707783,174,2017-11-30T19:45:00+00:00,3510,29927.876062,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30
1309172,1873823,174,2017-11-30T21:45:00+00:00,3510,15303.633895,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30
1309173,7376879,174,2017-12-03T06:45:00+00:00,3510,12752.003943,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03
1309174,1259696,174,2017-12-03T15:45:00+00:00,3510,13869.983259,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03


In [10]:
data = pd.merge(data, holiday_data, on=['SiteId', 'Date'], how='left')
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,ForecastPeriodNS,Date,Holiday
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-03,NaN
4,6124150,235,2014-01-01T15:45:00+00:00,5004,153731.391434,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7707783,174,2017-11-30T19:45:00+00:00,3510,29927.876062,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30,NaN
1309172,1873823,174,2017-11-30T21:45:00+00:00,3510,15303.633895,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30,NaN
1309173,7376879,174,2017-12-03T06:45:00+00:00,3510,12752.003943,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03,NaN
1309174,1259696,174,2017-12-03T15:45:00+00:00,3510,13869.983259,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03,NaN


In [11]:
data_merged = pd.merge(data, weather_data, on=['Timestamp', 'SiteId'], how='left')
data_merged

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,WednesdayIsDayOff,ThursdayIsDayOff,FridayIsDayOff,SaturdayIsDayOff,SundayIsDayOff,ForecastPeriodNS,Date,Holiday,Temperature,Distance
0,323604,235,2014-01-02T19:00:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN,NaN,NaN
1,2813181,235,2014-01-02T22:00:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN,NaN,NaN
2,4006999,235,2014-01-02T22:45:00+00:00,5004,155498.418922,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-02,NaN,NaN,NaN
3,106973,235,2014-01-03T00:30:00+00:00,5004,157265.446409,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-03,NaN,NaN,NaN
4,6124150,235,2014-01-01T15:45:00+00:00,5004,153731.391434,10906.977168,15.0,18.0,False,False,False,False,False,True,True,900000000000,2014-01-01,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326833,7707783,174,2017-11-30T19:45:00+00:00,3510,29927.876062,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30,NaN,NaN,NaN
1326834,1873823,174,2017-11-30T21:45:00+00:00,3510,15303.633895,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-11-30,NaN,NaN,NaN
1326835,7376879,174,2017-12-03T06:45:00+00:00,3510,12752.003943,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03,NaN,NaN,NaN
1326836,1259696,174,2017-12-03T15:45:00+00:00,3510,13869.983259,587.763238,15.0,18.0,False,False,False,False,False,True,True,3600000000000,2017-12-03,NaN,NaN,NaN


In [12]:
data_merged['Index'] = data_merged.groupby(['SiteId', 'Timestamp']).cumcount() + 1

left_cols = [col for col in data.columns]
data_pivot = data_merged.pivot(index=left_cols, columns='Index', values=['Temperature', 'Distance']).reset_index()

data_pivot.columns = [f'{col[0]}_{col[1]}' if col[0] not in left_cols else col[0] for col in data_pivot.columns]

data_pivot

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Date,Holiday,Temperature_1,Temperature_2,Temperature_3,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4
0,2,119,2015-01-05T18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,112,2015-11-08T15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,2015-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,22,2016-06-17T17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,2016-06-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,302,2013-01-11T18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,2013-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27,6,2013-04-18T17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,2013-04-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08T14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,2017-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309172,7868989,287,2016-10-15T13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,2016-10-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309173,7868992,120,2016-03-05T00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,2016-03-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1309174,7868993,27,2017-05-30T22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,2017-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def dataoveriew(data, message):
    print(f'{message}:\n')
    print('Number of rows: ', data.shape[0])
    print("\nNumber of features:", data.shape[1])
    print("\nData Features:")
    print(data.columns.tolist())
    print("\nMissing values:", data.isnull().sum().values.sum())
    print("\nUnique values:")
    print(data.nunique())

dataoveriew(data_pivot, 'Overview of the dataset')

Overview of the dataset:

Number of rows:  1309176

Number of features: 26

Data Features:
['obs_id', 'SiteId', 'Timestamp', 'ForecastId', 'Value', 'Surface', 'Sampling', 'BaseTemperature', 'MondayIsDayOff', 'TuesdayIsDayOff', 'WednesdayIsDayOff', 'ThursdayIsDayOff', 'FridayIsDayOff', 'SaturdayIsDayOff', 'SundayIsDayOff', 'ForecastPeriodNS', 'Date', 'Holiday', 'Temperature_1', 'Temperature_2', 'Temperature_3', 'Temperature_4', 'Distance_1', 'Distance_2', 'Distance_3', 'Distance_4']

Missing values: 11633766

Unique values:
obs_id               1309176
SiteId                   267
Timestamp             301774
ForecastId              6974
Value                 609436
Surface                  267
Sampling                   4
BaseTemperature            3
MondayIsDayOff             1
TuesdayIsDayOff            1
WednesdayIsDayOff          1
ThursdayIsDayOff           1
FridayIsDayOff             2
SaturdayIsDayOff           2
SundayIsDayOff             2
ForecastPeriodNS           3
Date   

In [14]:
fill_values = {
    'Temperature_1': data_pivot['Temperature_1'].mean(),  
    'Temperature_2': data_pivot['Temperature_2'].mean(),  
    'Temperature_3': data_pivot['Temperature_3'].mean(),  
    'Temperature_4': data_pivot['Temperature_4'].mean(),  
    'Distance_1': 0,  
    'Distance_2': 0,  
    'Distance_3': 0,  
    'Distance_4': 0,  
    'Holiday': ''
}
data = data_pivot.fillna(value=fill_values)
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Date,Holiday,Temperature_1,Temperature_2,Temperature_3,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4
0,2,119,2015-01-05T18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,2015-01-05,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
1,3,112,2015-11-08T15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,2015-11-08,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
2,4,22,2016-06-17T17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,2016-06-17,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
3,11,302,2013-01-11T18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,2013-01-11,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
4,27,6,2013-04-18T17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,2013-04-18,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08T14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,2017-11-08,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
1309172,7868989,287,2016-10-15T13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,2016-10-15,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
1309173,7868992,120,2016-03-05T00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,2016-03-05,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0
1309174,7868993,27,2017-05-30T22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,2017-05-30,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0


In [15]:
data['Date'] = pd.to_datetime(data['Date'])
data['DateInWeek'] = data['Date'].dt.day_name()
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Holiday,Temperature_1,Temperature_2,Temperature_3,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek
0,2,119,2015-01-05T18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Monday
1,3,112,2015-11-08T15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Sunday
2,4,22,2016-06-17T17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday
3,11,302,2013-01-11T18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday
4,27,6,2013-04-18T17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08T14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Wednesday
1309172,7868989,287,2016-10-15T13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday
1309173,7868992,120,2016-03-05T00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday
1309174,7868993,27,2017-05-30T22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Tuesday


In [16]:
bins = list(range(0, 25))
labels = ['{}-{}'.format(i, i + 1) for i in bins[:-1]]

In [17]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'])
data['Time'] = data['Timestamp'].dt.time
#data = data.drop(columns=['Timestamp'])
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Temperature_1,Temperature_2,Temperature_3,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek,Time
0,2,119,2015-01-05 18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Monday,18:30:00
1,3,112,2015-11-08 15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Sunday,15:30:00
2,4,22,2016-06-17 17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday,17:10:00
3,11,302,2013-01-11 18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday,18:30:00
4,27,6,2013-04-18 17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Thursday,17:45:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08 14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Wednesday,14:30:00
1309172,7868989,287,2016-10-15 13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday,13:30:00
1309173,7868992,120,2016-03-05 00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday,00:45:00
1309174,7868993,27,2017-05-30 22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,13.422958,12.948928,13.248485,10.087013,0.0,0.0,0.0,0.0,Tuesday,22:45:00


In [18]:
data['Hour'] = data['Time'].apply(lambda x: x.hour)
data['TimeInterval'] = pd.cut(data['Hour'], bins=bins, labels=labels, right=False)
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Temperature_3,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek,Time,Hour,TimeInterval
0,2,119,2015-01-05 18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Monday,18:30:00,18,18-19
1,3,112,2015-11-08 15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Sunday,15:30:00,15,15-16
2,4,22,2016-06-17 17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday,17:10:00,17,17-18
3,11,302,2013-01-11 18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Friday,18:30:00,18,18-19
4,27,6,2013-04-18 17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Thursday,17:45:00,17,17-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08 14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Wednesday,14:30:00,14,14-15
1309172,7868989,287,2016-10-15 13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday,13:30:00,13,13-14
1309173,7868992,120,2016-03-05 00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Saturday,00:45:00,0,0-1
1309174,7868993,27,2017-05-30 22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,13.248485,10.087013,0.0,0.0,0.0,0.0,Tuesday,22:45:00,22,22-23


In [19]:
def hour_to_part(hour):
    if (6<=hour<=12): return "Morning" 
    elif (12<hour<=18): return "Afternoon"
    elif (18<hour<=24): return "Evening"
    else: return "Night"
 
data['PartOfDay'] = data['Hour'].apply(hour_to_part)
data

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek,Time,Hour,TimeInterval,PartOfDay
0,2,119,2015-01-05 18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Monday,18:30:00,18,18-19,Afternoon
1,3,112,2015-11-08 15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Sunday,15:30:00,15,15-16,Afternoon
2,4,22,2016-06-17 17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Friday,17:10:00,17,17-18,Afternoon
3,11,302,2013-01-11 18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Friday,18:30:00,18,18-19,Afternoon
4,27,6,2013-04-18 17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Thursday,17:45:00,17,17-18,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08 14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Wednesday,14:30:00,14,14-15,Afternoon
1309172,7868989,287,2016-10-15 13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Saturday,13:30:00,13,13-14,Afternoon
1309173,7868992,120,2016-03-05 00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Saturday,00:45:00,0,0-1,Night
1309174,7868993,27,2017-05-30 22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Tuesday,22:45:00,22,22-23,Evening


In [20]:
df = data.dropna(subset=['Value'])
df

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek,Time,Hour,TimeInterval,PartOfDay
0,2,119,2015-01-05 18:30:00+00:00,2378,1971.344365,18648.464511,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Monday,18:30:00,18,18-19,Afternoon
1,3,112,2015-11-08 15:30:00+00:00,2276,876.615143,8554.793631,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Sunday,15:30:00,15,15-16,Afternoon
2,4,22,2016-06-17 17:10:00+00:00,703,15091.513995,7392.365415,10.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Friday,17:10:00,17,17-18,Afternoon
3,11,302,2013-01-11 18:30:00+00:00,6821,45430.633875,21284.578622,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Friday,18:30:00,18,18-19,Afternoon
4,27,6,2013-04-18 17:45:00+00:00,51,43082.633137,9150.195373,30.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Thursday,17:45:00,17,17-18,Afternoon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309171,7868980,63,2017-11-08 14:30:00+00:00,1511,12923.387161,1629.458932,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Wednesday,14:30:00,14,14-15,Afternoon
1309172,7868989,287,2016-10-15 13:30:00+00:00,6428,7998.454103,22495.507056,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Saturday,13:30:00,13,13-14,Afternoon
1309173,7868992,120,2016-03-05 00:45:00+00:00,2511,2959.836376,8793.378373,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Saturday,00:45:00,0,0-1,Night
1309174,7868993,27,2017-05-30 22:45:00+00:00,873,144635.916423,17.321667,15.0,18.0,False,False,...,10.087013,0.0,0.0,0.0,0.0,Tuesday,22:45:00,22,22-23,Evening


In [21]:
num_splits = len(df) // 100000 + 1
    
for i in range(num_splits):
    
    start_row = i * 100000
    end_row = (i + 1) * 100000
    
    split_df = df.iloc[start_row:end_row]

    #split_df.to_csv(f'../dataset/preprocessed-training-data/power-laws-forecasting-energy-consumption-preprocessed-training-data-split_{i + 1}.csv', index=False)
    split_df.to_csv(f'../dataset/preprocessed-test-data/power-laws-forecasting-energy-consumption-preprocessed-test-data-split_{i + 1}.csv', index=False)